# Создание и обучение модели LSTM на аккордах + длительностях

## Подключение библиотек

In [ ]:
import collections
import datetime
# import fluidsynth
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple

## Задаем стандарный random seed и проверяем GPU

In [ ]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

## Загружаем csv данные

In [ ]:
train = pd.read_csv('TrainFiltChordino.csv').drop('Unnamed: 0', axis = 1)
test = pd.read_csv('TestFiltChordino.csv').drop('Unnamed: 0', axis = 1)

## Конвертируем csv в numpy

In [ ]:
trainData = []
for i in train.index.values.tolist():
    row = list(train.iloc[i])
    trainData.append([[row[i], row[i+1]] for i in range(0,len(row),2)])
testData = []
for i in test.index.values.tolist():
    row = list(test.iloc[i])
    testData.append([[row[i], row[i+1]] for i in range(0,len(row),2)])

In [ ]:
testData = np.array(testData)
trainData = np.array(trainData)

In [ ]:
trainData = np.array([item for sublist in trainData for item in sublist])
testData = np.array([item for sublist in testData for item in sublist])

In [ ]:
key_order = ['chord', 'duration']

### Создаем tf.Dataset

In [ ]:
notes_ds = tf.data.Dataset.from_tensor_slices(trainData)
notes_ds.element_spec

In [ ]:
def create_sequences(
    dataset: tf.data.Dataset, 
    seq_length: int,
    vocab_size = 128,
) -> tf.data.Dataset:
    seq_length = seq_length+1

  # Take 1 extra for the labels
    windows = dataset.window(seq_length, shift=1, stride=1,
                              drop_remainder=True)
    print(type(windows))
  # `flat_map` flattens the" dataset of datasets" into a dataset of tensors
    flatten = lambda x: x.batch(seq_length, drop_remainder=True)
    sequences = windows.flat_map(flatten)

  # Normalize note pitch
    def scale_pitch(x):
        x = x/[vocab_size,1.0]
        return x

  # Split the labels
    def split_labels(sequences):
        inputs = sequences[:-1]
        labels_dense = sequences[-1]
        labels = {key:labels_dense[i] for i,key in enumerate(key_order)}

        return scale_pitch(inputs), labels

    return sequences.map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
seq_length = 50
vocab_size = 200
seq_ds = create_sequences(notes_ds, seq_length, vocab_size)
seq_ds.element_spec

### Проверка данных после преобразования

In [ ]:
for seq, target in seq_ds.take(1):
    print('sequence shape:', seq.shape)
    print('sequence elements (first 10):', seq[0: 10])
    print()
    print('target:', target)

### Кешируем данные для более быстрого доступа, также перемешиваем и разделяем на batch

In [ ]:
batch_size = 64
buffer_size = 10000 - seq_length  # the number of items in the dataset
train_ds = (seq_ds
            .shuffle(buffer_size)
            .batch(batch_size, drop_remainder=True)
            .cache()
            .prefetch(tf.data.experimental.AUTOTUNE))

### Написали свою функцию потерь, чтобы контролировать отрицательные значения

In [ ]:
def mse_with_positive_pressure(y_true: tf.Tensor, y_pred: tf.Tensor):
    mse = (y_true - y_pred) ** 2
    positive_pressure = 10 * tf.maximum(-y_pred, 0.0)
    return tf.reduce_mean(mse + positive_pressure)

## Инициализация модели и обучение

### Создаем модель загружаем функции ошибки, веса по параметрам и выводим данные о ней

In [ ]:
input_shape = (seq_length, 2)
learning_rate = 0.005

inputs = tf.keras.Input(input_shape)
x = tf.keras.layers.LSTM(128)(inputs)

outputs = {
  'chord': tf.keras.layers.Dense(193, name='chord')(x),
  'duration': tf.keras.layers.Dense(1, name='duration')(x),
}

model = tf.keras.Model(inputs, outputs)

loss = {
      'chord': tf.keras.losses.SparseCategoricalCrossentropy(
          from_logits=True),
      'duration': mse_with_positive_pressure,
}

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(loss=loss, optimizer=optimizer)

model.summary()

In [ ]:
losses = model.evaluate(train_ds, return_dict=True)
losses

In [ ]:
model.compile(
    loss=loss,
    loss_weights={
        'pitch': 0.5,
        'duration':2.0,
    },
    optimizer=optimizer,
)

In [ ]:
model.evaluate(train_ds, return_dict=True)

### Обучение модели

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='./training_checkpoints/ckpt_{epoch}',
        save_weights_only=True),
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=5,
        verbose=1,
        restore_best_weights=True),
]

In [ ]:
epochs = 50

history = model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
)

### Построение графика ошибки по эпохам

In [ ]:
plt.plot(history.epoch, history.history['loss'], label='total loss')
plt.show()

### Cозранение модели для будующего использования

In [ ]:
model.save("path/to/my_model.h5")